## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem # импортируем библиотеку
from collections import Counter # контейнер Counter из модуля collections.

In [2]:
data = pd.read_csv('/datasets/data.csv')
display(data)
display(data.info())
display(data.dtypes)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

### Вывод

***Тип данных указан не верно в столбцах 'education_id', 'family_status_id', 'debt'***
- Разный регистр в столбце education
- Отрицательные значения в столбце days_employed
- Леммы в значениях в столбце purpose***




### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
nan = data[data['total_income'].isnull()].head(20) # Поиск столбцов с NaN
data.loc[data['children'] < 0, 'children'] = data['children'] * -1 # Заменяем все отрицательные значения на положительные в столбце 'days_employed'
data.loc[data['days_employed'] < 0, 'days_employed'] = data['days_employed'] * -1
data.loc[data['children'] == 20 , 'children'] = 2 # Замена значения 20 на 2
days_employed_avg = data['days_employed'].median() # Медиана 
total_income_avg = data['total_income'].mean() # Средее значение
dob_years_avg = data['dob_years'].mean() # Медиана
data['days_employed'] = data['days_employed'].fillna(value = days_employed_avg) # замена пропусщеных значений на days_employed_avg
data['total_income'] = data['total_income'].fillna(value = total_income_avg) # замена пропусщеных значений на total_income_avg
data['dob_years'] = data['dob_years'].replace(0, dob_years_avg) # замена  значений на total_income_avg

### Вывод

***Были обнаружены пропущенные значения в столбцах 'total_income' и 'days_employed'***
- Возможно люди не указали свой доход и время работы хотя некоторые из их написали что сейчас работают.
- В столбце 'days_employed' были отрицательные значения. Возможно это ошибка появилась при выгрузке данных
- Пропуски я заполнены в первом столбце с помощью  медианы. Так-как значения сильно отлчались 
- Во втором с помощью среднего значения.

### Замена типа данных

In [4]:
data['total_income'] = data['total_income'].astype('int') # замена вещественного типа данных на целочисленный в столбце total_income
data['days_employed'] = data['days_employed'].astype('int') # замена вещественного типа данных на целочисленный в столбце days_employed
data.loc[data['income_type'] == 'предприниматель', 'income_type'] = 'компаньон' # замена значения 
data['dob_years'] = data['dob_years'].astype('int')
display(data)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

***Метод astype я выбрал так-как он подходит лучше всего в данном случае. Есть еще 1 метод to_numeric(),
но он перевождит строки в float.***



### Обработка дубликатов

In [5]:
print(data['education'].value_counts()) # находим выбросы
    # Удаление выбросов
data = data.drop(data[data.income_type == 'в декрете'].index) 
data = data.drop(data[data.income_type == 'безработный'].index)
data = data.drop(data[data.income_type == 'студент'].index)
data = data.drop(data[data.children == 5].index)
data = data.drop(data[data.education == 'ученая степень'].index)
data = data.drop(data[data.gender == 'XNA'].index)



среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


In [6]:
data['education'] = data['education'].str.lower() # Все строки к одному регистру 
data = data.drop_duplicates().reset_index(drop=True) # удаление дубликатов и сброс индексов 

### Вывод

**В таблице было не много дубликатов, но были слова с разным регистром, и несколько выбросов.**
- Использовал метод drop_duplicates()
- Возможно дубликаты появились из-за програмной ошибки.

### Лемматизация

In [7]:
m = Mystem()
purpose_lem = [] # создаем список 
for i in data['purpose']:
    lemmas = m.lemmatize(i) # Лемматизация
    purpose_lem.append(lemmas) # добавлем получсишшиеся значения в список
    
data['purpose_lem'] = purpose_lem # добавлем новую колонку в наш датафрейм 
print(data['purpose_lem'])


0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21431                  [операция,  , с,  , жилье, \n]
21432               [сделка,  , с,  , автомобиль, \n]
21433                              [недвижимость, \n]
21434    [на,  , покупка,  , свой,  , автомобиль, \n]
21435             [на,  , покупка,  , автомобиль, \n]
Name: purpose_lem, Length: 21436, dtype: object


### Вывод

In [8]:
lems = '' # создаем пустую строку
for lem in data['purpose_lem']: # Разбиваем дата сет на списки
    lems += ' '.join(lem) # Слепляем 
lems = lems.split() # Создаем строку 
print(Counter(lems))

Counter({'недвижимость': 6344, 'покупка': 5890, 'жилье': 4455, 'автомобиль': 4303, 'образование': 4011, 'с': 2915, 'операция': 2601, 'свадьба': 2323, 'свой': 2228, 'на': 2220, 'строительство': 1877, 'высокий': 1372, 'получение': 1314, 'коммерческий': 1310, 'для': 1288, 'жилой': 1229, 'сделка': 940, 'дополнительный': 906, 'заниматься': 902, 'проведение': 767, 'сыграть': 765, 'сдача': 650, 'семья': 638, 'собственный': 634, 'со': 626, 'ремонт': 606, 'подержанный': 485, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


***Из всех категорий я выделил 4 основные (недвижимость, жилье, автомобиль, образование, свадьба)***

### Категоризация данных

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
data['total_income'].quantile([0.25, 0.5, 0.75, 1]) #50%-ый квантиль == медиане

0.25     107629.00
0.50     156033.50
0.75     195759.75
1.00    2265604.00
Name: total_income, dtype: float64

In [10]:
"""
Возвращает уровень дохода по значению total_income используя правила:

- 'нищите' при значении income <= 50000

- 'бедные' при значении 50000 <= income <= 150000 включительно

- 'средний класс' при значении 150000 <= income <= 400000 включительно

- 'богатые' во всех остальных случаях
"""

def income_group(row): 

    income = row['total_income']


    if income <= 107629.00:
        return 'нищите'
    

    if 156033.50 <= income <= 195759.75:


        return 'бедные'
    
    if 195759.75 <= income <= 2265604.00:


        return 'средний класс'

    return 'богатые'
data['income_group'] = data.apply(income_group, axis=1) # создаем столбец с полученными значениями 

income_group_dict = data[['income_group', 'debt']] # берем из датафренйма только нужные толбцы
print(income_group_dict.groupby('income_group').mean().sort_values('debt',ascending=False)) # группируем по income_group и находим среднее debt

                   debt
income_group           
богатые        0.087330
бедные         0.086397
нищите         0.079306
средний класс  0.071469


In [11]:
# основное это были ли задолжности, а дальше к этому можно добавить любой столбец
children_dict = data[['children', 'debt']]
family_status_dict = data[['family_status', 'debt']]
total_income_dict = data[['total_income', 'debt']]
purpose_dict = data[['purpose', 'debt']]

# Добавленные категрии
children_total_income = data[['children', 'total_income']] 
income = data[['income_type', 'total_income']]
income_group_dict = data[['income_group', 'debt']]
status_dict = data[['family_status', 'dob_years']]

### Вывод

***Я выделил основные категории которые мне понадобились в последующих заданиях***


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
children_dict = data[['children', 'debt']] # берем из датафренйма только нужные толбцы
print(children_dict.groupby('children').mean().sort_values('debt',ascending=False)) # группируем по children и находим среднее debt

              debt
children          
4         0.097561
2         0.094499
1         0.091490
3         0.082067
0         0.075465


### Вывод

***Есть. Семьи без детей отдают кредиты лучше, в то время, как многодетные семьи возвращают долги хуже. Мне кажется это связано с тем что на ребенка уходит очень много времени и денег. Получается в семье появляется тот, кто тратит, но не зарабатывает. Так же родители не могут зарабатывать так же как без детей, потому что их нужно воспитывать.***



- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
family_status_dict = data[['family_status', 'debt']] # берем из датафренйма только нужные толбцы
print(family_status_dict.groupby('family_status').mean().sort_values('debt',ascending=False)) # группируем по family_status и находим среднее debt

                           debt
family_status                  
Не женат / не замужем  0.097544
гражданский брак       0.093562
женат / замужем        0.075363
в разводе              0.071189
вдовец / вдова         0.065693


### Вывод

***Те кто Не женат и не замужем а так же в гражданском браке чаще всего молодые, и плохо понимают как вообще работают кредиты, так-как финансовой грамнотности нету, или она на минимальном уровне пожэтому кредиты они возвращабт реже всех. Получается категория женат / замужем и  в разводе в основном люди среднего возраста и они более ответсвенные. В категории вдовец / вдова чаще всего встречаются пожилые они почти всега отдают деньги.*** 

In [14]:
# В подкреплене моей теории можно провести анализ данных 
status_dict = data[['family_status', 'dob_years']] # берем из датафренйма только нужные толбцы
print(status_dict.groupby('family_status').mean().sort_values('dob_years',ascending=False)) # группируем по family_status и находим среднее dob_years

                       dob_years
family_status                   
вдовец / вдова         56.727842
в разводе              45.885260
женат / замужем        43.720045
гражданский брак       42.292501
Не женат / не замужем  38.614454


In [15]:
income = data[['income_type', 'total_income']] # берем из датафренйма только нужные толбцы
print(income.groupby('income_type').mean().sort_values('total_income',ascending=False)) # группируем по income_type и находим среднее total_income

              total_income
income_type               
компаньон    199021.245814
госслужащий  170623.097595
сотрудник    161952.148880
пенсионер    140150.895507


### Вывод

***Богатые возвращают кредит реже чем бедные и даже нищщие. Возможно потому что большие деньги в основном зарабатывают предпрениматели, и кредит на бизнес обычно довольно большой и отдать его почти невозможно если дело прогорело. У бедных чаще всего просто нет денег чтоб вернуть, а вот к нищщим относяться в основном пенсионеры. Они более ответсвенные. Средний класс зарабатывает достаточно чтоб вернуть деньги банку и чаще всего у них стабильная работа.***

- Как разные цели кредита влияют на его возврат в срок?

In [16]:
"""
Возвращает тип кредита по значению purpose_lem используя правила:

- 'недвижимость' при значении ('жилье' in income) or ('недвижимость' in income)

- 'автомобиль' при значении 'автомобиль' in income

- 'свадьба' при значении 'свадьба' in income

- 'образование' при значении 'образование' in income:

- 'не понятно' во всех остальных случаях
"""
def purpose_group(row):
    income = row['purpose_lem']
    if ('жилье' in income) or ('недвижимость' in income):
        return 'недвижимость'
    
    if 'автомобиль' in income:


        return 'автомобиль'

    if 'свадьба' in income:


        return 'свадьба'

    if 'образование' in income:


        return 'образование'
    
    else:

        return 'не понятно'

        
data['purpose_group'] = data.apply(purpose_group, axis=1) # создаем столбец с полученными значениями
purpose_group_dict = data[['purpose_group', 'debt']] # берем из датафренйма только нужные толбцы
print(purpose_group_dict.groupby('purpose_group').mean().sort_values('debt',ascending=False)) # группируем по income_group и находим среднее debt

                   debt
purpose_group          
автомобиль     0.093423
образование    0.092246
свадьба        0.080069
недвижимость   0.072322


### Вывод

***Кредит на автомобиль возвращают реже всего потому что этот актив быстро теряет в цене, и требует довольно больших вложений, а многие не думают об этом при покупке. С образованием мне кажется людям чаще всего сложно и работать и учиться. На свадьбу чаще всего дарят деньги и семья может вместе погасить кредит, это не много проще чем делать это оджному, а недвижемость это та вещь за которую если не будешь платить просто выгонят, поэтому люди пытаются найти деньги где угодно чтоб погасить ипотеку***


### Шаг 4. Общий вывод

В среднем клиенты не возврают деньги примерно с одинаковой вероятностью.
Уровень дохода влияет сильнее всего на это. Так-же важной чертой является цель кредита, семейное положение и колдичество детей.
Семьи без детей отдают кредиты лучше, в то время, как многодетные семьи возвращают долги хуже. Те кто Не женат и не замужем а так же в гражданском браке чаще всего не отдают кредиты, а вот вдовец / вдова чаще всего отдают деньги. Богатые возвращают кредит реже чем бедные и даже нищщие. Средний класс зарабатывает достаточно чтоб вернуть деньги банку и обычно у них стабильная работа поэтому в этой категории они самые надежные заемщики. И Самые рискованные категории кредита это автомобиль и образование, а вот самые защищенные это недвижемость.
Портрет самого надежного заёмщика - У него нет детей, вдовец, с заработком от 195759.75 до 2265604 и пытающийся взять кредит на недвижемость.
Портрет самого рискованного заёмщика - У него 4 ребенка, не женат или в гражданском браке, и с доходом от 2265604 и пытается он взять кредит на машину или образование. 

In [17]:
#Cамые надежные заемщики 
print(data[(data['children'] == 0) & (data['family_status'] == 'вдовец / вдова') & ( data['total_income'] <= 2265604) & ( data['total_income'] >= 195759.75) & ( data['purpose_group'] == 'недвижимость')].head(60))

       children  days_employed  dob_years            education  education_id  \
767           0         367177         59              среднее             1   
902           0           1826         66              среднее             1   
1383          0         353802         37              среднее             1   
1387          0           1985         56               высшее             0   
1455          0         384834         58               высшее             0   
1915          0         373836         67               высшее             0   
1925          0         378929         65              среднее             1   
2466          0           1160         62               высшее             0   
2649          0         364303         62              среднее             1   
2894          0           1272         58              среднее             1   
3023          0            367         44               высшее             0   
3075          0           1554         5

In [18]:
# Самые рискованные заемщики
print(data[(data['children'] == 4) & (data['family_status'] == 'Не женат / не замужем') & ( data['total_income'] >= 2265604) & ( data['purpose_group'] == 'автомобиль')].head(60))

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, purpose_lem, income_group, purpose_group]
Index: []


***К сожалению, а может и к счатью таких в татасете нет***